In [2]:
#importing libraries
import numpy as np
import pandas as pd

In [3]:
df=pd.read_csv(r'D:\University\AI_diploma\Technocolabs\fina\modelling_data.csv')
df.drop(labels='Unnamed: 0',axis=1,inplace=True)
df.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,status
0,0.159229,-0.562613,-0.591805,-0.157964,0.659951,0.282487,-0.708394,1
1,0.025409,-0.545583,-0.628715,0.360402,-0.721383,-0.049150,0.534937,1
2,-1.181523,4.085948,1.818133,0.832832,1.918683,-0.973053,-0.373344,2
3,0.041926,-0.528871,-0.607936,0.369352,-0.656534,-0.026201,0.592552,1
4,-1.295015,4.279216,2.080705,1.216708,2.516421,-0.710619,-1.778471,2


In [6]:
from sklearn.model_selection import train_test_split
from sklearn import naive_bayes
from sklearn.preprocessing import MinMaxScaler 
from sklearn.metrics import classification_report
from collections import Counter

In [16]:
X=df.drop(labels = 'status', axis = 1)
X2=X.copy()
y=df.status
scaler=MinMaxScaler()
X2 = scaler.fit_transform(X2)
X2 = pd.DataFrame(X2, columns = ['PC1','PC2','PC3','PC4','PC5','PC6','PC7'])
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X2_train,X2_test,y2_train,y2_test = train_test_split(X2, y, test_size=0.2, random_state=42)
print(Counter(y))

Counter({1: 38986, 2: 2783, 3: 1183, 4: 409})


In [18]:
#using multinomial naive bayes(with min-max scaled features as it doesn't accept negative values so normalized values can't
#be used)
MNB = naive_bayes.MultinomialNB()
MNB.fit(X2_train,y2_train)
y_hat=MNB.predict(X2_test)
print('Train accuracy is : {}'.format(MNB.score(X2_train,y2_train)))
print(classification_report(y2_test,y_hat))

Train accuracy is : 0.8979185885608856
              precision    recall  f1-score   support

           1       0.90      1.00      0.95      7839
           2       0.00      0.00      0.00       526
           3       0.00      0.00      0.00       223
           4       0.00      0.00      0.00        85

    accuracy                           0.90      8673
   macro avg       0.23      0.25      0.24      8673
weighted avg       0.82      0.90      0.86      8673



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
#using complemented naive bayes(also with min-max scaled features)
CNB = naive_bayes.ComplementNB()
CNB.fit(X2_train,y2_train)
y_hat=CNB.predict(X2_test)
CNB.score(X2_train,y2_train)
print(classification_report(y2_test,y_hat))

              precision    recall  f1-score   support

           1       0.99      1.00      0.99      7839
           2       0.69      0.99      0.82       526
           3       0.00      0.00      0.00       223
           4       0.00      0.00      0.00        85

    accuracy                           0.96      8673
   macro avg       0.42      0.50      0.45      8673
weighted avg       0.94      0.96      0.95      8673



In [21]:
#using gaussian naive bayes( with normalized features)
GNB = naive_bayes.GaussianNB()
GNB.fit(X_train,y_train)
y_hat=GNB.predict(X_test)
GNB.score(X_train,y_train)
print(classification_report(y_test,y_hat))

              precision    recall  f1-score   support

           1       0.99      0.97      0.98      7839
           2       0.85      0.79      0.82       526
           3       0.60      0.63      0.62       223
           4       0.10      0.33      0.16        85

    accuracy                           0.95      8673
   macro avg       0.64      0.68      0.64      8673
weighted avg       0.96      0.95      0.95      8673



### we can notice that f1-score is very bad for all types of naive bayes algorithms. as we have strongly biased labels, so we can try oversampling methods that works for multiclassification problems 

#### let's try smote oversampling

In [23]:
from imblearn.over_sampling import SMOTE
oversample=SMOTE()
X_over, y_over = oversample.fit_resample(X, y)
X2_over,y2_over = oversample.fit_resample(X2, y)
print(Counter(y_over))


Counter({1: 38986, 2: 38986, 3: 38986, 4: 38986})


In [25]:
#trying modelling again and seeing the best model of them (with normalized for gauss, min-max scaled data for the rest of models)
X_train,X_test,y_train,y_test = train_test_split(X_over, y_over, test_size=0.2, random_state=42)
GNB = naive_bayes.GaussianNB()
GNB.fit(X_train,y_train)
y_hat=GNB.predict(X_test)
GNB.score(X_train,y_train)
print('Train accuracy is : {}'.format(GNB.score(X_train,y_train)))
print(classification_report(y_test,y_hat))

Train accuracy is : 0.6899763536531602
              precision    recall  f1-score   support

           1       0.63      0.94      0.75      7672
           2       0.78      0.57      0.66      7896
           3       0.68      0.81      0.74      7852
           4       0.77      0.45      0.56      7769

    accuracy                           0.69     31189
   macro avg       0.71      0.69      0.68     31189
weighted avg       0.71      0.69      0.68     31189



In [27]:

X2_train,X2_test,y2_train,y2_test = train_test_split(X2_over, y2_over, test_size=0.2, random_state=42)
CNB = naive_bayes.ComplementNB()
CNB.fit(X2_train,y2_train)
y_hat=CNB.predict(X2_test)
CNB.score(X2_train,y2_train)
print('Train accuracy is : {}'.format(CNB.score(X2_train,y2_train)))
print(classification_report(y2_test,y_hat))

MNB = naive_bayes.MultinomialNB()
MNB.fit(X2_train,y2_train)
y_hat=MNB.predict(X2_test)
print('Train accuracy is : {}'.format(MNB.score(X2_train,y2_train)))
print(classification_report(y2_test,y_hat))

Train accuracy is : 0.6186204961724981
              precision    recall  f1-score   support

           1       0.98      0.13      0.23      7672
           2       0.71      0.61      0.66      7896
           3       0.57      0.74      0.64      7852
           4       0.58      0.98      0.73      7769

    accuracy                           0.62     31189
   macro avg       0.71      0.62      0.57     31189
weighted avg       0.71      0.62      0.57     31189

Train accuracy is : 0.7411326199350727
              precision    recall  f1-score   support

           1       0.73      0.86      0.79      7672
           2       0.70      0.74      0.72      7896
           3       0.69      0.68      0.69      7852
           4       0.89      0.69      0.78      7769

    accuracy                           0.74     31189
   macro avg       0.75      0.74      0.74     31189
weighted avg       0.75      0.74      0.74     31189



In [28]:
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks

In [30]:
#trying another combined-sampling method
resample=SMOTETomek(tomek=TomekLinks(sampling_strategy='majority'))
X_over, y_over = resample.fit_resample(X, y)
X2_over, y2_over = resample.fit_resample(X2, y)
X_train,X_test,y_train,y_test = train_test_split(X_over, y_over, test_size=0.2, random_state=42)
X2_train,X2_test,y2_train,y2_test = train_test_split(X2_over, y2_over, test_size=0.2, random_state=42)

In [31]:
print(Counter(y_over))

Counter({2: 38986, 3: 38986, 4: 38986, 1: 38945})


In [32]:
GNB = naive_bayes.GaussianNB()
GNB.fit(X_train,y_train)
y_hat=GNB.predict(X_test)
GNB.score(X_train,y_train)
print('Train accuracy is : {}'.format(MNB.score(X_train,y_train)))
print(classification_report(y_test,y_hat))

Train accuracy is : 0.6767450810602781
              precision    recall  f1-score   support

           1       0.62      0.94      0.75      7668
           2       0.77      0.58      0.66      7762
           3       0.68      0.79      0.74      7814
           4       0.77      0.45      0.57      7937

    accuracy                           0.69     31181
   macro avg       0.71      0.69      0.68     31181
weighted avg       0.71      0.69      0.68     31181



In [33]:
CNB = naive_bayes.ComplementNB()
CNB.fit(X2_train,y2_train)
y_hat=CNB.predict(X2_test)
CNB.score(X2_train,y_train)
print('Train accuracy is : {}'.format(CNB.score(X2_train,y2_train)))
print(classification_report(y2_test,y_hat))

MNB = naive_bayes.MultinomialNB()
MNB.fit(X2_train,y2_train)
y_hat=MNB.predict(X2_test)
print('Train accuracy is : {}'.format(MNB.score(X2_train,y2_train)))
print(classification_report(y2_test,y_hat))

Train accuracy is : 0.6211815076730649
              precision    recall  f1-score   support

           1       0.99      0.15      0.26      7624
           2       0.72      0.63      0.67      7796
           3       0.57      0.74      0.65      7815
           4       0.59      0.99      0.74      7946

    accuracy                           0.63     31181
   macro avg       0.72      0.62      0.58     31181
weighted avg       0.72      0.63      0.58     31181

Train accuracy is : 0.7352832699924632
              precision    recall  f1-score   support

           1       0.69      0.87      0.77      7624
           2       0.71      0.75      0.73      7796
           3       0.69      0.69      0.69      7815
           4       0.91      0.63      0.74      7946

    accuracy                           0.73     31181
   macro avg       0.75      0.73      0.73     31181
weighted avg       0.75      0.73      0.73     31181



##### the best model is multinomial for our min-max scaled oversampled data with the first smote oversampling method with accuracy 74.2%